In [1]:
import numpy as np
from qiskit import transpile
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA,SLSQP, POWELL
from qiskit.primitives import Estimator
from qiskit_algorithms.utils import algorithm_globals
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.quantum_info import Statevector, SparsePauliOp
import scipy
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit.circuit import ParameterVector
import numpy as np
from pyscf import gto, scf
import pyscf.mcscf as mcscf

In [2]:
#Create the all possible excitations
num_spartial_orbital = 4
num_spin_orbitals = num_spartial_orbital * 2

# Initialize the mapper
mapper = JordanWignerMapper()
#Create an identity operator
I = FermionicOp({'': 1.0}, num_spin_orbitals =num_spin_orbitals)
I = mapper.map(I)

 #list of occupied orbitals
occupied = []
for i in range(num_spin_orbitals//4):
    occupied.append(i)
    occupied.append(i+num_spin_orbitals//2)
#occupied = [0,4,1,5]
    
# Generate all possible single excitations
excitations = []
def all_excitations(num_spin_orbitals):
    for i in range(num_spin_orbitals):
        for j in range(i+1, num_spin_orbitals):
            # Prevent electrons from moving from alpha spin to beta spin and beta spin to alpha
            if i != j and ((i < num_spin_orbitals // 2 and j < num_spin_orbitals // 2) or (i >= num_spin_orbitals // 2 and j >= num_spin_orbitals // 2)):
                # Only consider excitations where the first two alpha and beta spins are filled with electrons
                if (i in occupied and j not in occupied): 
                    excitation = FermionicOp({f'+_{j} -_{i}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                    excitations.append(excitation)
    
        #Generate possible double excitations
        #Double excitations all from alpha or beta orbitals
            for k in range(j+1, num_spin_orbitals):
                for l in range(k+1, num_spin_orbitals):
                    if i != j and k != l and ((i < num_spin_orbitals // 2 and j < num_spin_orbitals // 2 and k < num_spin_orbitals // 2 and l < num_spin_orbitals // 2) or (i >= num_spin_orbitals // 2 and j >= num_spin_orbitals // 2 and k >= num_spin_orbitals // 2 and l >= num_spin_orbitals // 2)):
                        # Only consider excitations where the first two alpha and beta spins are filled with electrons
                        if (i in occupied and k not in occupied and j in occupied and l not in occupied): 
                            excitation = FermionicOp({f'+_{l} +_{k} -_{i} -_{j}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                            excitations.append(excitation)
  
    for i in range(num_spin_orbitals // 2):
        for j in range(num_spin_orbitals // 2, num_spin_orbitals):
            for k in range(num_spin_orbitals // 2):
                for l in range(num_spin_orbitals // 2, num_spin_orbitals):
                    if i != k and j != l and i < k and j < l:
                        # Condition to ensure one alpha and one beta excitation
                        if (i in occupied and k not in occupied and j in occupied and l not in occupied): 
                           # Create the FermionicOp and add to double_exc list
                            exc = FermionicOp({f'+_{l} +_{k} -_{i} -_{j}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                            excitations.append(exc)

    return excitations
excitations = all_excitations(num_spin_orbitals)
print(len(excitations))
print(excitations)

26
[FermionicOp({'+_3 +_2 -_0 -_1': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_2 -_0': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_3 -_0': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_2 -_1': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_3 -_1': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_6 -_4 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_2 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_2 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_3 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_3 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_2 -_0 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_2 -_0 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_3 -_0 -_5': 1.0}, num_spin_orbitals=8, ), Fermio

In [ ]:
eigval = []
distances = np.linspace(0.60, 2.46, 19)
for d in distances:
    atom = f"N 0.0 0.0 0.0; H {d} 0.0 0.0; H -0.506, 0.876 0.0; H -0.506 -0.876 0.0"
    basis = "sto-6g"
    driver = PySCFDriver(atom=atom,  basis=basis)

    problem = driver.run()

    active_electrons = 4
    active_transformer = ActiveSpaceTransformer(num_electrons=active_electrons, num_spatial_orbitals=num_spartial_orbital)
    active_problem = active_transformer.transform(problem)

    seed = 170
    algorithm_globals.random_seed = seed

    mol = gto.M(atom = atom, basis=basis, verbose = 0)
    mf = scf.RHF(mol).run()
    mc = mcscf.CASCI(mf, ncas=num_spartial_orbital, nelecas=active_electrons)
    mc.kernel()

    frozen_core_energy = mc.e_tot - mc.e_cas
    core = frozen_core_energy
    #print("Frozen-core energy =", core)


    # Initialize the mapper
    mapper = JordanWignerMapper()
            
    # Map the electronic problem to a qubit operator
    qubit_op = mapper.map(active_problem.hamiltonian.second_q_op())
            
    # Initialize the UCCSD ansatz with Hartree-Fock initial state
    ansatz = UCCSD(
        active_problem.num_spatial_orbitals,
        active_problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            active_problem.num_spatial_orbitals,
            active_problem.num_particles,
            mapper
        ),
    )

    vqe = VQE(Estimator(), ansatz, SLSQP())
    vqe.initial_point = np.zeros(ansatz.num_parameters)
    nr = active_problem.nuclear_repulsion_energy 
    # Calculate the exact energy
    #creating a ground state eigensolver(vqe)
    #print("NR", nr)
    solver = GroundStateEigensolver(mapper, vqe)
    result = solver.solve(active_problem)
    print(f"Computing for bond length: {d:.2f} Å")

    gr = result.total_energies

    print(f"ground state energy = {gr}")
    # Extract the ground state wavefunction parameters

    psi_vqe = result.raw_result.optimal_point

    
    ansatz = ansatz.assign_parameters(dict(zip(ansatz.parameters, psi_vqe)), inplace=False)

    from qiskit_aer import AerSimulator
    simulator = AerSimulator(method='statevector')
    qc = transpile(ansatz, simulator)
    qc.save_statevector()

    aer_result = simulator.run(qc).result()
    statevector = aer_result.get_statevector(qc)

    
    # Initialize the matrix M
    num_excitations = len(excitations)
    M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
    S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
    # Compute the matrix elements
    for i in range(len(excitations) +1):
        for j in range(len(excitations)+1):
            G_i = excitations[i-1]
            G_j = excitations[j-1]
            op_i = mapper.map(G_i)
            op_j = mapper.map(G_j)
            op = op_i.adjoint()@qubit_op@op_j
            oj = qubit_op@op_j
            oi = op_i.adjoint()@qubit_op
                    
            if i == j == 0:
                M[i, j] = Statevector(statevector).expectation_value(qubit_op)
                S[i, j] = 1.0
            elif i==0 and j > 0:
                M[i, j] = Statevector(statevector).expectation_value(oj)
                S[i, j] = Statevector(statevector).expectation_value(op_j)
            elif i>0 and j==0:
                M[i, j] = Statevector(statevector).expectation_value(oi)
                S[i, j] = Statevector(statevector).expectation_value(op_i.adjoint())
            else:
                M[i, j] = Statevector(statevector).expectation_value(op)
                S[i, j] = Statevector(statevector).expectation_value(op_i.adjoint()@op_j)                      

    cond_num = np.linalg.cond(S)
    print("condition number:", cond_num)

    eig, ev = scipy.linalg.eigh(M, S) 
    eigval.append(eig + core)
    print("Eigenvalues from QSE:", eigval)

/var/folders/1s/dd2h77ks4wdccclxjmvsvk0h75nm_0/T/ipykernel_48251/396403616.py:45: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(Estimator(), ansatz, SLSQP())
